In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/amazonreviews/test.ft.txt.bz2
/kaggle/input/amazonreviews/train.ft.txt.bz2


In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import bz2



import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.python.keras import models, layers, optimizers
import tensorflow
from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
import bz2
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
import re

%matplotlib inline


In [5]:
def get_labels_and_texts(file):
    labels = []
    texts = []
    for line in bz2.BZ2File(file):
        x = line.decode("utf-8")
        labels.append(int(x[9]) - 1)
        texts.append(x[10:].strip())
    return np.array(labels), texts

In [6]:
labels=[]
for line in bz2.BZ2File("/kaggle/input/amazonreviews/test.ft.txt.bz2"):
    x=line.decode("utf-8")
    print(x[9])
    labels.append(int(x[9]) - 1)
    break
print(labels)

2
[1]


In [7]:
train_labels, train_texts = get_labels_and_texts('/kaggle/input/amazonreviews/train.ft.txt.bz2')
test_labels, test_texts = get_labels_and_texts("/kaggle/input/amazonreviews/test.ft.txt.bz2")

In [8]:
import re
NON_ALPHANUM = re.compile(r'[\W]')
NON_ASCII = re.compile(r'[^a-z0-1\s]')
def normalize_texts(texts):
    normalized_texts = []
    for text in texts:
        lower = text.lower()
        no_punctuation = NON_ALPHANUM.sub(r' ', lower)
        no_non_ascii = NON_ASCII.sub(r'', no_punctuation)
        normalized_texts.append(no_non_ascii)
    return normalized_texts
        
train_texts = normalize_texts(train_texts)
test_texts = normalize_texts(test_texts)
train_texts[1]

'the best soundtrack ever to anything   i m reading a lot of reviews saying that this is the best  game soundtrack  and i figured that i d write a review to disagree a bit  this in my opinino is yasunori mitsuda s ultimate masterpiece  the music is timeless and i m been listening to it for years now and its beauty simply refuses to fade the price tag on this is pretty staggering i must say  but if you are going to buy any cd for this much money  this is the only one that i feel would be worth every penny '

In [9]:
train_texts[14]

'awful beyond belief   i feel i have to write to keep others from wasting their money  this book seems to have been written by a th grader with poor grammatical skills for her age  as another reviewer points out  there is a misspelling on the cover  and i believe there is at least one per chapter  for example  it was mentioned twice that she had a  lean  on her house  i was so distracted by the poor writing and weak plot  that i decided to read with a pencil in hand to mark all of the horrible grammar and spelling  please don t waste your money  i too  believe that the good reviews must have been written by the author s relatives  i will not put much faith in the reviews from now on '

In [10]:
from sklearn.model_selection import train_test_split
train_texts, val_texts, train_labels, val_labels = train_test_split(
    train_texts, train_labels, random_state=42, test_size=0.2)

In [11]:
print(len(train_texts))
print(len(val_texts))

2880000
720000


In [12]:
import tensorflow
from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence

MAX_FEATURES = 12000
tokenizer = Tokenizer(num_words=MAX_FEATURES)
tokenizer.fit_on_texts(train_texts)
train_texts = tokenizer.texts_to_sequences(train_texts)
val_texts = tokenizer.texts_to_sequences(val_texts)
test_texts = tokenizer.texts_to_sequences(test_texts)

In [13]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
MAX_LENGTH = max(len(train_ex) for train_ex in train_texts)
train_texts = pad_sequences(train_texts, maxlen=MAX_LENGTH)
val_texts = pad_sequences(val_texts, maxlen=MAX_LENGTH)
test_texts = pad_sequences(test_texts, maxlen=MAX_LENGTH)

In [14]:
print(train_texts.shape,
val_texts.shape,
test_texts.shape)

(2880000, 255) (720000, 255) (400000, 255)


In [15]:
from tensorflow.python.keras import models, layers, optimizers
import tensorflow

def build_rnn_model():
    sequences = layers.Input(shape=(MAX_LENGTH,))
    embedded = layers.Embedding(MAX_FEATURES, 64)(sequences)
    x = layers.CuDNNGRU(128, return_sequences=True)(embedded)
    x = layers.CuDNNGRU(128)(x)
    x = layers.Dense(32, activation='relu')(x)
    x = layers.Dense(100, activation='relu')(x)
    predictions = layers.Dense(1, activation='sigmoid')(x)
    model = models.Model(inputs=sequences, outputs=predictions)
    model.compile(
        optimizer='rmsprop',
        loss='binary_crossentropy',
        metrics=['binary_accuracy']
    )
    return model
    
rnn_model = build_rnn_model()

In [16]:
rnn_model.fit(
    train_texts, 
    train_labels, 
    batch_size=128,
    epochs=1,
    validation_data=(val_texts, val_labels), )

2023-02-15 12:46:48.572111: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 2937600000 exceeds 10% of free system memory.


22499/22500 [============================>.] - ETA: 0s - loss: 0.1607 - binary_accuracy: 0.9390

2023-02-15 12:59:40.897286: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 734400000 exceeds 10% of free system memory.


22500/22500 [==============================] - 843s 37ms/step - loss: 0.1607 - binary_accuracy: 0.9390 - val_loss: 0.1345 - val_binary_accuracy: 0.9500
